 TODO(Grisha): does it belong to the `devops` dir?

# Description

The notebook reports the latest build status for multiple repos.

# Imports

In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import logging
from typing import Dict

import pandas as pd
from IPython.display import Markdown, display

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import helpers.lib_tasks_gh as hlitagh

In [14]:
hdbg.init_logger(verbosity=logging.INFO)
_LOG = logging.getLogger(__name__)
_LOG.info("%s", henv.get_system_signature()[0])
hprint.config_notebook()

INFO  # Git
  branch_name='CmampTask7128_Buildmeister_Dashboard_add_a_timestamp'
  hash='03ea809ed'
  # Last commits:
    * 03ea809ed Juraj Smeriga Fix wrong column name (#7127)                                     (22 minutes ago) Wed Feb 7 17:37:50 2024  (HEAD -> CmampTask7128_Buildmeister_Dashboard_add_a_timestamp, origin/master, origin/HEAD, origin/CmampTask7128_Buildmeister_Dashboard_add_a_timestamp, master)
    * 663f1e37a Juraj Smeriga Add new revision - bid ask resampled table schema changed (#7124) (   2 hours ago) Wed Feb 7 16:08:32 2024           
    * 6dd725d85 Sonaal   im_v2/common/notebooks/CmampTask7008_Understand_relationship_between_files_in_refinitiv_bucket.py (#7022) (   4 hours ago) Wed Feb 7 14:09:49 2024           
# Machine info
  system=Linux
  node name=7ae739d73976
  release=5.15.0-1052-aws
  version=#57~20.04.1-Ubuntu SMP Mon Jan 15 17:04:56 UTC 2024
  machine=x86_64
  processor=x86_64
  cpu count=8
  cpu freq=scpufreq(current=2499.998, min=0.0, max=0.0)
  me

In [15]:
# Set the display options to print the full table.
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# Utils

In [16]:
def make_clickable(url: str) -> str:
    """
    Wrapper to make the URL value clickable.

    :param url: URL value to convert
    :return: clickable URL link
    """
    return f'<a href="{url}" target="_blank">{url}</a>'


def color_format(val: str, status_color_mapping: Dict[str, str]) -> str:
    """
    Return the color depends on status.

    :param val: value of the status e.g. `failure`
    :param status_color_mapping: mapping statuses to the colors e.g.:
    ```
    {
       "success": "green",
       "failure": "red",
    }
    ```
    """
    if val in status_color_mapping:
        color = status_color_mapping[val]
    else:
        color = "grey"
    return f"background-color: {color}"

# GH workflows state

In [17]:
repo_list = [
    "cryptokaizen/cmamp",
    "cryptokaizen/orange",
    "cryptokaizen/lemonade",
    "sorrentum/dev_tools",
    "sorrentum/sorrentum",
]
workflow_df = hlitagh.gh_get_details_for_all_workflows(repo_list)
# Reorder columns.
columns_order = ["repo_name", "workflow_name", "conclusion", "url"]
workflow_df = workflow_df[columns_order]
# Make URL values clickable.
workflow_df["url"] = workflow_df["url"].apply(make_clickable)
_LOG.info(hpandas.df_to_str(workflow_df, log_level=logging.INFO))

WARNING * Failed assertion *
'0'
==
'2'
Continuing as per user request with only_warning=True
WARNING Not enough runs to compute status for 'Build dev image', repo 'cryptokaizen/cmamp', skipping the workflow
WARNING * Failed assertion *
'0'
==
'2'
Continuing as per user request with only_warning=True
WARNING Not enough runs to compute status for 'Build production cmamp image', repo 'cryptokaizen/cmamp', skipping the workflow
WARNING * Failed assertion *
'0'
==
'2'
Continuing as per user request with only_warning=True
WARNING Not enough runs to compute status for 'Check if the Linter was run', repo 'cryptokaizen/cmamp', skipping the workflow
WARNING * Failed assertion *
'0'
==
'2'
Continuing as per user request with only_warning=True
WARNING Not enough runs to compute status for 'Run linter', repo 'cryptokaizen/cmamp', skipping the workflow
WARNING * Failed assertion *
'0'
==
'2'
Continuing as per user request with only_warning=True
WARNING Not enough runs to compute status for 'Build i

,repo_name,workflow_name,conclusion,url
0,cryptokaizen/cmamp,Allure fast tests,success,"<a href=""https://github.com/cryptokaizen/cmamp/actions/runs/7808568279"" target=""_blank"">https://github.com/cryptokaizen/cmamp/actions/runs/7808568279</a>"
1,cryptokaizen/cmamp,Allure slow tests,success,"<a href=""https://github.com/cryptokaizen/cmamp/actions/runs/7808548104"" target=""_blank"">https://github.com/cryptokaizen/cmamp/actions/runs/7808548104</a>"
2,cryptokaizen/cmamp,Allure superslow tests,success,"<a href=""https://github.com/cryptokaizen/cmamp/actions/runs/7808557004"" target=""_blank"">https://github.com/cryptokaizen/cmamp/actions/runs/7808557004</a>"
,...,...,...,...
16,sorrentum/dev_tools,Superslow tests,success,"<a href=""https://github.com/sorrentum/dev_tools/actions/runs/7770587591"" target=""_blank"">https://github.com/sorrentum/dev_tools/actions/runs/7770587591</a>"
17,sorrentum/dev_tools,Update amp sumbodule,success,"<a href=""https://github.com/sorrentum/dev_tools/actions/runs/7808302374"" target=""_blank"">https://github.com/sorrentum/dev_tools/actions/runs/7808302374</a>"
18,sorrentum/sorrentum,Fast tests,failure,"<a href=""https://github.com/sorrentum/sorrentum/actions/runs/7808102957"" target=""_blank"">https://github.com/sorrentum/sorrentum/actions/runs/7808102957</a>"


INFO  None


In [18]:
status_color_mapping = {
    "success": "green",
    "failure": "red",
}
repos = workflow_df["repo_name"].unique()
display(Markdown("## Overall Status"))
current_timestamp = pd.Timestamp.now(tz="America/New_York")
display(Markdown(f"**Last run: {current_timestamp}**"))
for repo in repos:
    # Calculate the overall status.
    repo_df = workflow_df[workflow_df["repo_name"] == repo]
    overall_status = hlitagh.gh_get_overall_build_status_for_repo(repo_df)
    display(Markdown(f"## {repo}: {overall_status}"))
    repo_df = repo_df.drop(columns=["repo_name"])
    display(
        repo_df.style.map(
            color_format,
            status_color_mapping=status_color_mapping,
            subset=["conclusion"],
        )
    )

## Overall Status

**Last run: 2024-02-07 12:59:57.554240-05:00**

## cryptokaizen/cmamp: <span style="color:red">Failed</span>

,workflow_name,conclusion,url
0,Allure fast tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7808568279
1,Allure slow tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7808548104
2,Allure superslow tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7808557004
3,Fast tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7818893418
4,Import cycles detector,success,https://github.com/cryptokaizen/cmamp/actions/runs/7807768465
5,Run and publish the Buildmeister Dashboard,success,https://github.com/cryptokaizen/cmamp/actions/runs/7818452319
6,Slow tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7818893415
7,Superslow tests,success,https://github.com/cryptokaizen/cmamp/actions/runs/7807877868
8,Test coverage,failure,https://github.com/cryptokaizen/cmamp/actions/runs/7777681857


## cryptokaizen/orange: <span style="color:red">Failed</span>

,workflow_name,conclusion,url
9,Fast tests,success,https://github.com/cryptokaizen/orange/actions/runs/7814543113
10,Release new ECS preprod task definition,success,https://github.com/cryptokaizen/orange/actions/runs/7814428030
11,Slow tests,success,https://github.com/cryptokaizen/orange/actions/runs/7814543128
12,Superslow tests,failure,https://github.com/cryptokaizen/orange/actions/runs/7807841880
13,Update amp sumbodule,failure,https://github.com/cryptokaizen/orange/actions/runs/7808544691


## sorrentum/dev_tools: <span style="color:green">Success</span>

,workflow_name,conclusion,url
14,Fast tests,success,https://github.com/sorrentum/dev_tools/actions/runs/7808397959
15,Slow tests,success,https://github.com/sorrentum/dev_tools/actions/runs/7808332144
16,Superslow tests,success,https://github.com/sorrentum/dev_tools/actions/runs/7770587591
17,Update amp sumbodule,success,https://github.com/sorrentum/dev_tools/actions/runs/7808302374


## sorrentum/sorrentum: <span style="color:red">Failed</span>

,workflow_name,conclusion,url
18,Fast tests,failure,https://github.com/sorrentum/sorrentum/actions/runs/7808102957


# Allure reports

- fast tests: http://172.30.2.44/allure_reports/cmamp/fast/latest/index.html
- slow tests: http://172.30.2.44/allure_reports/cmamp/slow/latest/index.html
- superslow tests: http://172.30.2.44/allure_reports/cmamp/superslow/latest/index.html

# Number of open pull requests

In [19]:
for repo in repo_list:
    number_prs = len(hlitagh.gh_get_open_prs(repo))
    _LOG.info("%s: %s", repo, number_prs)

INFO  cryptokaizen/cmamp: 30
INFO  cryptokaizen/orange: 18
INFO  sorrentum/dev_tools: 2
INFO  sorrentum/sorrentum: 9


# Code coverage HTML-page

http://172.30.2.44/html_coverage/runner_master/